In [1]:
## Get FTD-Relevant Stock Symbols. Test Machine-Learning Model on list 

In [2]:
import pandas as pd

import glob

from pathlib import Path
import csv

In [3]:
# Initial imports
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import model_from_json
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler

%matplotlib inline

In [4]:
## Load pickle for exports and imports of data  
import pickle 
def load_obj(path):
    with open(path, 'rb') as f:
        return pickle.load(f)
    
def save_obj(obj, path ):
    with open(path, 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

In [5]:
path = Path('../Resources/finished_data_dict.pkl')
data_dict_import = load_obj(path)

In [6]:
path = Path('../Resources/finished_key_list.csv')
key_list_import_df = pd.read_csv(path,index_col=0)
key_list = []
for i in key_list_import_df['Symbol']:
    key_list.append(i)
len(key_list)

9992

In [7]:
# index_value = 0
# for i in range(len(key_list)):
#     if key_list[i] == symbol:
#         index_value = i
#         pass
# dict_key = key_list[index_value]
# df = data_dict_import[dict_key]['dataFrame'].copy()
# X = df.drop(columns='close').values
# y = y = df['close'].values

# scaler = StandardScaler().fit(X)
# X = scaler.transform(X)    

# num_of_inputs = 17
# num_of_outputs= 1

In [8]:
index_value = key_list[30]
index_value

'ABEO'

In [9]:
data_dict_import[index_value]['ftd_stats']

{'total_ftds_all': 9133837.0,
 'total_ftds_2021': 3259432.0,
 'total_ftds_ytd': 3485439.0,
 'os_ftd_pct_all': 8.961954421476875,
 'os_ftd_pct_2021': 3.1980952828371274,
 'os_ftd_pct_ytd': 3.419849232785514,
 'float_ftd_pct_all': 10.004079358277348,
 'float_ftd_pct_2021': 3.5699801070359207,
 'float_ftd_pct_ytd': 3.817520320806562,
 'short_volume_NSDQpct_all_data': 46.18597206353908,
 'shortexempt_volume_NSDQpct_all_data': 0.8019058154265507,
 'short_volume_NSDQpct_ytd': 45.36767248482744,
 'shortexempt_volume_NSDQpct_ytd': 0.7065807963817634,
 'short_volume_FMPpct_all_data': 19.690261418074723,
 'shortexempt_volume_FMPpct_all_data': 0.3418729634335912,
 'short_volume_FMPpct_ytd': 22.418312199610863,
 'shortexempt_volume_FMPpct_ytd': 0.34915498239045917}

In [10]:
data_dict_import[index_value]['companyProfile']['isEtf']

False

In [11]:
ftd_key_list = []
etf_key_list = []
eqt_key_list = []
float_count_is_0 = 0

for i in key_list:
    index_value = i
    float_ftd_pct_ytd = data_dict_import[index_value]['ftd_stats']['float_ftd_pct_ytd']
    ## Check if there's float data. Otherwise, calculate using Outstanding share data
    ## ETFs have no float data, so they always use Outstanding Shares 
    if float_ftd_pct_ytd == 0:        
        os_ftd_pct_ytd = data_dict_import[index_value]['ftd_stats']['os_ftd_pct_ytd']
        if os_ftd_pct_ytd >= 5:
            float_count_is_0 += 1
            ftd_key_list.append(i)
            isEtf = data_dict_import[index_value]['companyProfile']['isEtf']
            if isEtf == True:
                etf_key_list.append(i)
            else:
                eqt_key_list.append(i)
    else:
        if float_ftd_pct_ytd >= 5:
            ftd_key_list.append(i)
            isEtf = data_dict_import[index_value]['companyProfile']['isEtf']
            if isEtf == 'True':
                etf_key_list.append(i)
            else:
                eqt_key_list.append(i)

print(f'Float = 0 for '+str(float_count_is_0)+' symbols')
print('All list length:',len(ftd_key_list))
print('ETF list length:',len(etf_key_list))
print('EQT list length:',len(eqt_key_list))

Float = 0 for 2112 symbols
All list length: 3695
ETF list length: 1872
EQT list length: 1823


In [14]:
## Make a function to run this kind of model 

def mean_squared(symbol,validation_split=0.3,epochs=800,layers=1):
    index_value = 0
    for i in range(len(key_list)):
        if key_list[i] == symbol:
            index_value = i
            pass
    dict_key = key_list[index_value]
    df = data_dict_import[dict_key]['dataFrame'].copy()
    X = df.drop(columns='close').values
    y = df['close'].values
    
    scaler = StandardScaler().fit(X)
    X = scaler.transform(X)    
    
    num_of_inputs = 17
    num_of_outputs= 1

    
    ## Create Neural Network 
    if layers == 1:
        # Create a shallow, 1 hidden layer, neural network
        nn1 = Sequential()

        # Hidden layer
        nn1.add(Dense(units=8, input_dim=num_of_inputs, activation="relu"))

        # Output layer
        nn1.add(Dense(units=num_of_outputs, activation="linear"))

        # Compile the model
        nn1.compile(loss="mean_squared_error", optimizer="adam", metrics=["mse"])

        # Fit the model
        model_1 = nn1.fit(X, y, validation_split=validation_split, epochs=epochs,verbose=0)
        
        # Save model_1 as JSON
        nn_json1 = nn1.to_json()

        file_path = Path("../Resources/model_1_temp.json")
        with open(file_path, "w") as json_file:
            json_file.write(nn_json1)

        # Save weights
        file_path = "../Resources/model_1_temp.h5"
        nn1.save_weights(file_path)
        
        # load json and create model
        file_path = Path("../Resources/model_1_temp.json")
        with open(file_path, "r") as json_file:
            model_json = json_file.read()
        loaded_model1 = model_from_json(model_json)

        # load weights into new model
        file_path = "../Resources/model_1_temp.h5"
        loaded_model1.load_weights(file_path)
        
        # Make some predictions with the loaded model
        df1 = df.copy()
        df1["predicted"] = loaded_model1.predict(X)
        df_pred = df1[['close','predicted']]
        df_pred
        
    elif layers == 2:
        # Define the model - deep neural network with two layers
        nn2 = Sequential()

        # First hidden layer
        nn2.add(Dense(units=8, input_dim=num_of_inputs, activation="relu"))

        # Second hidden layer
        nn2.add(Dense(units=8, activation="relu"))

        # Output layer
        nn2.add(Dense(units=num_of_outputs, activation="linear"))
        
        # Compile the model
        nn2.compile(loss="mean_squared_error", optimizer="adam", metrics=["mse"])

        # Fit the model
        model_2 = nn2.fit(X, y, validation_split=0.3, epochs=800, verbose=0)
        
        # Save model_2 as JSON
        nn_json2 = nn2.to_json()

        file_path = Path("../Resources/model_2_temp.json")
        with open(file_path, "w") as json_file:
            json_file.write(nn_json2)

        # Save weights
        file_path = "../Resources/model_2_temp.h5"
        nn2.save_weights(file_path)
        
        # load json and create model
        file_path = Path("../Resources/model_2_temp.json")
        with open(file_path, "r") as json_file:
            model_json = json_file.read()
        loaded_model2 = model_from_json(model_json)

        # load weights into new model
        file_path = "../Resources/model_2_temp.h5"
        loaded_model2.load_weights(file_path)
        
        # Make some predictions with the loaded model
        df2 = df.copy()
        df2["predicted"] = loaded_model2.predict(X)
        df_pred = df2[['close','predicted']]
        df_pred
        
    return df_pred

In [16]:
test_model = mean_squared('GME',layers=2)
test_model

,close,predicted
Date,,
2016-01-04,28.309999,28.295673
2016-01-05,28.770000,28.671391
2016-01-06,28.370001,28.235249
2016-01-07,28.450001,28.406145
2016-01-08,28.370001,28.296890
...,...,...
2021-10-25,173.970000,175.240433
2021-10-26,177.840000,181.110794
2021-10-27,173.510000,175.669449
